## Tour of ROS tools with examples

ROS has plenty of tools that can help prototype, test and deploy robots. 

We will focus of some of the most important:

 - Rviz: which is a tool to visialize the state of the robot in 3D as well as to see its sensor readouts

 - Tf: which is used to simplify calculations of robot's kinematics

 - Rosbag: which is used to log Robot experiments



### Rviz

Rviz is a visualization tool for ROS which means that it is very good in understanding standard ROS messagess and parameters and translating them into understandable visualizations. 

Normally you don't program the Rviz itself but send information to appropriete topics and parameters. You can, however write additional toolboxes for it. 



In [7]:
# we launch the rviz using the command rosrun rviz rviz
import roslaunch

import rospy

import rosparam




In [14]:
rospy.init_node("tourist")

In [6]:
### we will put robot description of Pi Robot (mymodelrobot.appspot.com) to the parameter server

with open("../../description/pi_robot.urdf", "r") as pi_robot_file:
    urdf_text= pi_robot_file.read()


rosparam.set_param_raw("robot_description",urdf_text)

In [7]:
### we can now view the robot in rviz

launch = roslaunch.scriptapi.ROSLaunch()
launch.start()

node_rviz = roslaunch.core.Node("rviz",
                            "rviz", name="rviz") # this starts a ROS Node from ipython_robot_prototyping 
node_state_publisher= roslaunch.core.Node("robot_state_publisher", "robot_state_publisher", name="robot_state_publisher") #this starts our turtle

node_joint_publisher = roslaunch.core.Node("joint_state_publisher","joint_state_publisher", name="joint_state_publisher")

nodes=[node_rviz,node_state_publisher,node_joint_publisher]

started roslaunch server http://igor-laptop-linux:58642/

SUMMARY

PARAMETERS
 * /rosdistro: b'indigo\n'
 * /rosversion: b'1.11.21\n'

NODES

ROS_MASTER_URI=http://igor-laptop-linux:11311
core service [/rosout] found


In [8]:
### this will start rviz and two helper programs, robot_state publisher and robot_joint_publisher
all_procesess=[launch.launch(node) for node in nodes]
    

process[rviz-1]: started with pid [24461]
process[robot_state_publisher-2]: started with pid [24462]
process[joint_state_publisher-3]: started with pid [24463]


The code above started 3 programs:

 1. rviz which is the visualizer
 2. robot_state_publisher which takes joint_state messages and using robot design from URDF from "robot_description" describes where each robot part should be ( forward kinematics)
 3. joint_state_publisher which is a helper node that just publishes that our joints are in 0 position
 
To see our robot in Rviz, you need to click Add and then select "Robot Model"

![](images/add_robot_rviz.png)

Also change the "fixed frame" to base_link 
You should see a Pi robot model in Rviz.

Normally we would get the state of the robot -- its joint positions/velocities/ accelerations from its sensors of from a simulation. We can still play around by publishing additional joint state messages

In [36]:
all_procesess[2].stop() #first we need to stop joint_state_publisher

In [15]:
from sensor_msgs.msg import JointState
from std_msgs.msg import Header

pub = rospy.Publisher('joint_states', JointState, queue_size=10)

joint_names= ['torso_joint', 
'head_pan_joint', 'head_tilt_joint', 'left_shoulder_forward_joint',
'right_shoulder_forward_joint', 'left_shoulder_up_joint', 
'right_shoulder_up_joint', 'left_elbow_joint', 
'right_elbow_joint', 'left_wrist_joint', 'right_wrist_joint']
    
    

In [16]:
joint_msg = JointState()
joint_msg.header = Header()
joint_msg.header.stamp = rospy.Time.now()
joint_msg.name = joint_names
joint_msg.position = [0.5,0,0,0,0,0,0,0,0,0,0]
joint_msg.velocity = []
joint_msg.effort = []

pub.publish(joint_msg)

This sends new joint position to our robot simulation, moving the torso.

We can send such position in a loop

In [17]:
def move_torso(angle):
    joint_msg = JointState()
    joint_msg.header = Header()
    joint_msg.header.stamp = rospy.Time.now()
    joint_msg.name = joint_names
    joint_msg.position = [angle,0,0,0,0,0,0,0,0,0,0]
    joint_msg.velocity = []
    joint_msg.effort = []

    pub.publish(joint_msg)

In [58]:
### we can create a small loop that moves shoulder 
rate = rospy.Rate(10) # 10Hz move 10 times per s

import math

### amplitude - how far torso will go
### omega - how fast torso will go

amplitude= math.pi/2
omega= 1.5
while not rospy.is_shutdown():
        t=rospy.Time.now().to_sec()
        move_torso(amplitude*math.sin(omega*t))
        rate.sleep()

KeyboardInterrupt: 

### Excercise

Try to also move a shoulder 

In [ ]:
### 

We can add additional elements to Rviz scene, such as markers

In [51]:
from visualization_msgs.msg import Marker

In [53]:
vis_pub = rospy.Publisher("visualization_marker",Marker, queue_size=5 );

In [57]:
marker = Marker()
marker.header.frame_id = "base_link";
marker.header.stamp = rospy.Time.now()
marker.ns = "markers";
marker.id = 0;
marker.type = 2 #sphere
marker.action =0 # add
marker.pose.position.x = 0.5;
marker.pose.position.y = 0.5;
marker.pose.position.z =0.5;
marker.pose.orientation.x = 0.0;
marker.pose.orientation.y = 0.0;
marker.pose.orientation.z = 0.0;
marker.pose.orientation.w = 1.0;
marker.scale.x = 0.1;
marker.scale.y = 0.1;
marker.scale.z = 0.1;
marker.color.a = 1.0; # Don't forget to set the alpha!
marker.color.r = 0.0;
marker.color.g = 1.0;
marker.color.b = 0.0;

vis_pub.publish( marker );

To see the result in Rviz you need to add new "Marker" display type. We can orbit around the robot similarely how you rotated the shoulder.

### Excercise

Orbit the sphere around the robot

HINT: Create a function for x y similar to that

x = 0.5 sin( t* omega)

y = 0.5 cos (t * omega)

### Other types of information

You can also see other types of information like 3d point clouds or camera streams. We will use usb_camera package to stream our images to ROS.


In [68]:
camera_node=roslaunch.core.Node("cv_camera", "cv_camera_node", name="cv_camera") #this starts our turtle
nodes.append(camera_node)
all_procesess.append(launch.launch(camera_node))

ERROR: cannot launch node of type [cv_camera/cv_camera_node]: cv_camera
ROS path [0]=/opt/ros/indigo/share/ros
ROS path [1]=/home/igor/catkin_ws/src
ROS path [2]=/home/igor/projects/shadow_robot/base/src
ROS path [3]=/home/igor/projects/shadow_robot/base_deps/src
ROS path [4]=/opt/ros/indigo/share
ROS path [5]=/opt/ros/indigo/stacks


RLException: failed to launch cv_camera/cv_camera_node

This time you need to add "Image" and select topic as "usb_camera/image_raw"

### TF

Calculating forward kinematics is one of most frequent things in robotics.

We want to know how each object (say robot) is oriented relative to some other object (say a door handle)
so we can then calculate how to move one towards another.

Because of that coordinate frames (TF -- transform frames) that describe how elements are related is an important object.

Currentely a set of coordinate frames is published for us by robot_state_publisher. Let's read one such frame


In [12]:
from tf2_msgs.msg import TFMessage

In [13]:
tf_message=rospy.wait_for_message("/tf",TFMessage)
print(tf_message)

transforms: 
  - 
    header: 
      seq: 0
      stamp: 
        secs: 1502820842
        nsecs: 302270720
      frame_id: head_link
    child_frame_id: antenna_link
    transform: 
      translation: 
        x: 0.0
        y: -0.025
        z: 0.065
      rotation: 
        x: 0.0
        y: 0.0
        z: 0.0
        w: 1.0
  - 
    header: 
      seq: 0
      stamp: 
        secs: 1502820842
        nsecs: 302280984
      frame_id: base_link
    child_frame_id: base_laser
    transform: 
      translation: 
        x: 0.18
        y: 0.0
        z: 0.07
      rotation: 
        x: 0.0
        y: 0.0
        z: 0.0
        w: 1.0
  - 
    header: 
      seq: 0
      stamp: 
        secs: 1502820842
        nsecs: 302283957
      frame_id: base_link
    child_frame_id: cpu_link
    transform: 
      translation: 
        x: 0.025
        y: 0.0
        z: 0.085
      rotation: 
        x: 0.0
        y: 0.0
        z: 0.0
        w: 1.0
  - 
    header: 
      seq: 0
      stamp: 
 

As usuall, there is a lot of info in such a single message. But the important part are two:
one is the child - parent relation between frame_id and child_frame_id. 

/tf topic has messages that are *relations* between some coordinate systems. In can be realition between a ground and a robot but it can be a relation between a robot torso and a robot shoulder. To build a whole robot structure we collect a set of such relations to build a *kinematic tree*.

Let's see how this kinematic tree looks for our robot

In [29]:
!rosrun tf view_frames && evince frames.pdf

Listening to /tf for 5.000000 seconds
Done Listening
dot - graphviz version 2.36.0 (20140111.2315)

Detected dot version 2.36
frames.pdf generated



But actually we normally do not use these messagages directely, instead we use tf.TransformListener'ers (instead of Subscribers) and tf.TransformBroadcaster's. This is  because we can select some particular transformations that we need and the listener will calculate them for us. 

For example, let's say we want to know the position of robot's head relative to its base

In [41]:
move_torso(0)

In [42]:
%%python2


import tf
import rospy
## here we just move robot to some base position

rospy.init_node('turtle_tf_listener')

listener = tf.TransformListener()


print("test")
rate = rospy.Rate(10.0)

try:

        (trans,rot) = listener.lookupTransform( '/torso_link','right_shoulder_link', rospy.Time(0))
except (tf.LookupException, tf.ConnectivityException, tf.ExtrapolationException) as e:
        print(e)
        print("fufuf")
        
print(trans)
    

test
"torso_link" passed to lookupTransform argument target_frame does not exist. 
fufuf


Traceback (most recent call last):
  File "<stdin>", line 22, in <module>
NameError: name 'trans' is not defined


In [63]:
trans

NameError: name 'trans' is not defined